In [2]:
from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader
from network_models.soundsream_models_and_utils.ss_complex_conv_net import SSComplexConvModel3Sec

batch_size = 6

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncs_betterSS_12_1_relwrap_music.pkl", device="cuda")


modelDimRed = SSDimRedModel(eval_mode= True).cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175, eval_mode= True).cuda()
modelComplexConv = SSComplexConvModel3Sec(eval_mode=True).to("cuda")

trainDS, testDs = train_val_dataset(data_set, val_split=0.1, seed=100)
trainDS, valDs = train_val_dataset(trainDS, val_split=0.1, seed=100)

2023-02-22 17:59:10.224841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 17:59:10.677940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 17:59:10.678016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 17:59:10.678021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
import gc

gc.collect()

#modelConv.load_state_dict(torch.load("../content/soundstream/experiments/experiment_v_12_1_NR3_relu/Run_Nr_0/conv/emo_reco_best_ep852_acc_75.pth"))
#modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/experiment_v_12_1_NR3_relu/Run_Nr_0/dimred/emo_reco_best_ep824_acc_70.pth"))
modelDimRed.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/dim_red/Nr1_tess/emo_reco_best_ep105_acc_96.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr2_tess/emo_reco_best_ep92_acc_100.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr4/emo_reco_best_ep95_acc_69.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr2_tess/emo_reco_best_ep92_acc_100.pth"))


<All keys matched successfully>

In [4]:
from utils.utils__evalutation import evaluate
import gc


def evaluateLoc(device, model, dataset):

    return evaluate(device, model, dataset, batch_size)

In [7]:
from utils.eval_utils import classificationReport, confusion_matrix
trueConv, predsConv = evaluateLoc("cuda", modelDimRed, data_set)
print("------------------ Conv: ----------------")
classificationReport(trueConv, predsConv, data_set.encoded_dataset.label_list)
confusion_matrix(trueConv, predsConv, data_set.encoded_dataset.label_list)

print()

0 of 6092
60 of 6092
120 of 6092
180 of 6092
240 of 6092
300 of 6092
360 of 6092
420 of 6092
480 of 6092
540 of 6092
600 of 6092
660 of 6092
720 of 6092
780 of 6092
840 of 6092
900 of 6092
960 of 6092
1020 of 6092
1080 of 6092
1140 of 6092
1200 of 6092
1260 of 6092
1320 of 6092
1380 of 6092
1440 of 6092
1500 of 6092
1560 of 6092
1620 of 6092
1680 of 6092
1740 of 6092
1800 of 6092
1860 of 6092
1920 of 6092
1980 of 6092
2040 of 6092
2100 of 6092
2160 of 6092
2220 of 6092
2280 of 6092
2340 of 6092
2400 of 6092
2460 of 6092
2520 of 6092
2580 of 6092
2640 of 6092
2700 of 6092
2760 of 6092
2820 of 6092
2880 of 6092
2940 of 6092
3000 of 6092
3060 of 6092
3120 of 6092
3180 of 6092
3240 of 6092
3300 of 6092
3360 of 6092
3420 of 6092
3480 of 6092
3540 of 6092
3600 of 6092
3660 of 6092
3720 of 6092
3780 of 6092
3840 of 6092
3900 of 6092
3960 of 6092
4020 of 6092
4080 of 6092
4140 of 6092
4200 of 6092
4260 of 6092
4320 of 6092
4380 of 6092
4440 of 6092
4500 of 6092
4560 of 6092
4620 of 6092
4680 o

In [14]:
from utils.eval_utils import classificationReport, confusion_matrix

trueConv, predsConv = evaluateLoc("cuda", modelConv, testDs)
trueDimRed, predsDimRed = evaluateLoc("cuda", modelDimRed, testDs)

print("------------------ Conv: ----------------")
classificationReport(trueConv, predsConv, data_set.encoded_dataset.label_list)
confusion_matrix(trueConv, predsConv, data_set.encoded_dataset.label_list)
print("------------------ DimRed: ----------------")
classificationReport(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)
confusion_matrix(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)


print("")

0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
------------------ Conv: ----------------
           accuracy  precision  recall   support
   angry     0.746     0.705     0.798    99
 disgust     0.746     0.826     0.710    107
    fear     0.746     0.756     0.775    80
   happy     0.746     0.611     0.714    77
 neutral     0.746     0.787     0.779    95
     sad     0.746     0.833     0.714    91
surprise     0.746     0.710     0.721    61
                                          610

 
     avg     0.746     0.747     0.745    

                   angry     disgust        fear       happy     neutral         sad    surprise    support
       angry       0.798       0.040       0.030       0.061       0.020       0.010       0.040         99
     disgust       0.112       0.710       0.009